In [1]:
#---------------------------------------------Libs import------------------------------------#

from datetime import datetime
import psycopg2
import pandas as pd
import os
from ultralytics import YOLO
import cv2
from datetime import datetime

#---------------------------------------------Database connection------------------------------------#

# Connect with PostgreSQL
def connect_to_database():
    conn = psycopg2.connect(
        dbname="******",
        user="******",
        password="******",
        host="localhost"
    )
    return conn

# Read data from table
def fetch_data(conn):
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM sinks;")
    data = cursor.fetchall()
    column_names = [desc[0] for desc in cursor.description]
    cursor.close()
    return data, column_names

#---------------------------------------------Object detection----------------------------------------#

def perform_object_detection():
    VIDEOS_DIR = os.path.join('.', 'videos')

    video_path = os.path.join(VIDEOS_DIR, 'No_Dokos.mp4')
    video_path_out = '{}_out.mp4'.format(video_path)

    cap = cv2.VideoCapture(video_path)
    ret, frame = cap.read()
    H, W, _ = frame.shape
    out = cv2.VideoWriter(video_path_out, cv2.VideoWriter_fourcc(*'MP4V'), int(cap.get(cv2.CAP_PROP_FPS)), (W, H))

    model_path = os.path.join('.', 'runs', 'detect', 'train3', 'weights', 'best.pt')

    # Load a model
    model = YOLO(model_path)  # load a custom model
    lst = []
    threshold = 0.7

    while ret:

        results = model(frame)[0]

        for result in results.boxes.data.tolist():
            x1, y1, x2, y2, score, class_id = result

            if score > threshold:
                lst.append(results.names[int(class_id)])
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 255), 2)
                cv2.putText(frame, results.names[int(class_id)].upper(), (int(x1), int(y1 - 10)),
                            cv2.FONT_HERSHEY_TRIPLEX, 0.7, (225, 126, 0), 2)

        out.write(frame)
        ret, frame = cap.read()

    cap.release()
    out.release()
    cv2.destroyAllWindows()

    return lst

#-------------------------------------------Results processing------------------------------------------#

def process_results(lst, df):
    no_comma = []
    for i in set(lst):
        x = i.replace(",", "")
        no_comma.append(x)

    last_available_sink = []
    no_available = []
    need_to_lay_out = {}

    for articul in df['articul'].unique():
            available_count = df[df['articul'] == articul]['amount'].sum()

            if articul not in no_comma and available_count >= 1:
                need_to_lay_out[articul] = available_count

            elif articul not in no_comma and available_count < 1:
                no_available.append(articul)

            elif articul in no_comma and available_count == 1:
                last_available_sink.append(articul)

    return need_to_lay_out, last_available_sink, no_available

#------------------------------------------------------------Creating file with results--------------------------------#

def create_result_file(need_to_lay_out, last_available_sink, no_available):
    # Set the current date for the file name
    current_date = datetime.now().strftime("%Y-%m-%d")

    # Opening a file for writing
    with open(f"{current_date}.txt", "w") as file:
        # Sinks that are in stock but not on display
        file.write(f"{10*'-'} Раковини, які є в наявності, але немає на вітрині {10*'-'}\n\n")
        for articul, count in need_to_lay_out.items():
            file.write(f"- {articul} | {count}шт\n-------------------------\n")

        # Sinks that remained only in the showcase version
        file.write(f"\n\n{10*'-'} Раковини, які залишились тільки в вітринному варіанті {10*'-'}\n\n")
        for articul in last_available_sink:
            file.write(f"- {articul}\n")

        # Sinks that are out of stock
        file.write(f"\n\n{10*'-'} Раковини, яких немає в наявності {10*'-'}\n\n")
        for articul in no_available:
            file.write(f"- {articul}\n")
        file.write("\n")

#-----------------------------------------------------------Main function--------------------------------------#

def main():
    # Підключення до бази даних
    conn = connect_to_database()

    # Зчитування даних з таблиці
    data, column_names = fetch_data(conn)
    df = pd.DataFrame(data, columns=column_names)  # Використання іменованих колонок

    # Закриваємо з'єднання з базою даних
    conn.close()
    
    # Виконання розпізнавання об'єктів
    lst = perform_object_detection()

    # Обробка результатів
    need_to_lay_out, last_available_sink, no_available = process_results(lst, df)

    # Створення файлу з результатами
    create_result_file(need_to_lay_out, last_available_sink, no_available)

    
if __name__ == "__main__":
    main()


0: 384x640 1 FEME-CE-10-001,, 95.0ms
Speed: 6.0ms preprocess, 95.0ms inference, 1126.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 FEME-CE-10-001,, 106.4ms
Speed: 2.0ms preprocess, 106.4ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 FEME-CE-10-001,, 47.8ms
Speed: 2.0ms preprocess, 47.8ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 FEME-CE-10-001,, 45.5ms
Speed: 1.0ms preprocess, 45.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 FEME-CE-10-001,, 48.8ms
Speed: 2.0ms preprocess, 48.8ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 FEME-CE-10-001,, 48.8ms
Speed: 1.0ms preprocess, 48.8ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 FEME-CE-10-001,, 48.8ms
Speed: 1.0ms preprocess, 48.8ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 FEME-CE-10-001,, 49.8ms
Speed: 1.0ms